In [89]:
import pandas as pd
import os

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

import numpy as np
from tkinter import messagebox

import re
import time

In [90]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

path = filedialog.askopenfilename()

正在打开……


In [91]:
sheetName=['城市','绝地反击型','强力进攻型','提值进攻型','稳存进攻型']

In [102]:
def  rank_handle(sheetName):
    file1 = pd.read_excel(path, sheet_name=sheetName,skiprows=1)

    group_col=['区县',	'片区',	'片区ID',	'服务中心',	'服务中心ID', 	'中心属性',	'中心类型',	'中心分级']

    col_name=[]
    for col in file1.columns:
        for col1 in group_col:
            if col1 in col:
                col_name.append(col)
        # for col2 in col:
            # print(col2)
        if '得分' in col  and '修正得分' not in col and '类型得分' not in col and '两比得分' not in col and '三比得分' not in col and '排名得分' not in col and '最终得分' not in col and '实际服务中心人数' not in col:
                col_name.append(col)
        
    f1=file1[col_name]

    # print(f1)
    for col in col_name:
        flag=0
        for col1 in group_col:
      
            if col1 in col:
                flag+=1
                continue
        if flag==0:
       
            f1[col+'排名']=f1[col].rank(method='dense',ascending=False)


    res_col=group_col
    for col in f1.columns:
        if '排名' in col:
            res_col.append(col)
    # print(res_col)
    # print(f1)
    f1_res=f1[res_col]
    # print(f1_res)
    # f1_res.columns=res_col
    return f1_res

In [117]:
group_col_str='区县|片区|片区ID|服务中心|服务中心ID|中心属性|中心类型|中心分级'

In [146]:
%%capture --no-display
df_res=[]
df_res_ss=[]
for s in sheetName:
 
    df_tmp=rank_handle(s)
    
    col=df_tmp.columns[~df_tmp.columns.str.contains(group_col_str)]
    df_res_col=pd.DataFrame(columns=['类型'],data=[s])

    for c in col:
        data_sort=df_tmp.sort_values(by=[c],ascending=[False])
        
        grouped=data_sort['服务中心'].head(3)
        ss=''
        for v in grouped:
            # print(v)
            ss=ss+v+'\n'
        df_res_col[c]=ss
        # print(df_res_col)

    # print(col)
    df_res.append(df_tmp)
    df_res_ss.append(df_res_col)

### ***将df合并至一个子表sheet里***

In [147]:
def df_to_oneSheet(df_list,name):
    # Create a list of DataFrames
    # Save the DataFrame list to an excel file with one sheet
    with pd.ExcelWriter(name+'data.xlsx') as writer:
        startrow = 0
        for i, df in enumerate(df_list):
            df.to_excel(writer, sheet_name='sheet1', startrow=startrow, index=False)
            startrow += df.shape[0] + 2
            # Add an empty row
            writer.save()


In [148]:
df_to_oneSheet(df_res,'排名')
df_to_oneSheet(df_res_ss,'结论')